In [2]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("dataset.csv")

# Display first few rows
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [3]:
# List all columns to understand data structure
print(df.columns.tolist())

['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']


In [4]:
# Hyperparameters
alpha = 2.0           # Controls price increment sensitivity
base_price = 10.0     # Starting price for each lot
min_price = 5.0       # Minimum allowed price
max_price = 20.0      # Maximum allowed price


In [5]:
# Create a working copy
df_pricing = df.copy()

# Add a new price column
df_pricing["price"] = np.nan

# Get unique lot identifiers from 'SystemCodeNumber'
parking_ids = df_pricing["SystemCodeNumber"].unique()

# Initialize the latest price for each parking lot
latest_prices = {pid: base_price for pid in parking_ids}


In [6]:
# Apply pricing model row by row
for idx, row in df_pricing.iterrows():
    lot_id = row["SystemCodeNumber"]
    occupancy = row["Occupancy"]
    capacity = row["Capacity"]

    # Handle capacity=0 edge case
    if capacity == 0:
        price = latest_prices[lot_id]
    else:
        price_increment = alpha * (occupancy / capacity)
        price = latest_prices[lot_id] + price_increment

    # Smooth price between limits
    price = np.clip(price, min_price, max_price)

    # Save price
    df_pricing.at[idx, "price"] = price

    # Update price for next timestep
    latest_prices[lot_id] = price


In [7]:
# Save the new DataFrame with pricing
df_pricing.to_csv("baseline_pricing_output.csv", index=False)

print("✅ Pricing complete. Results saved to 'baseline_pricing_output.csv'")


✅ Pricing complete. Results saved to 'baseline_pricing_output.csv'
